In [95]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os

# print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd
import altair as alt
import arrow

import src.severa.base_client
import src.severa.fetch
import src.severa.client
import src.severa.process
import src.database
from src.daterange import DateRange
import src.stable_hash
import datetime

reload(src.severa.base_client)
reload(src.severa.fetch)
reload(src.severa.client)
reload(src.severa.process)
reload(src.database)
reload(src.stable_hash)

span = DateRange(540)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
t = arrow.utcnow()
span = DateRange(t.shift(days=-60), t.shift(days=30))

span_past, span_future = span.cut(arrow.utcnow())

async with src.severa.client.Client() as f:
    # hours = await f.fetch_hours(span)
    # salesval = await f.fetch_salesvalue(span)
    billing = await f.fetch_billing(span)
    projs = await f.fetch_projects_with_cache()

2023-06-19 08:15:38.199 | DEBUG    | src.severa.base_client:get_with_retries:119 - Going too fast; sleeping 0.67s.
2023-06-19 08:15:38.885 | SUCCESS  | src.severa.base_client:get_with_retries:141 - HTTP/2 GET projects [0]: Response (100) in 0.31s.
2023-06-19 08:15:39.536 | DEBUG    | src.severa.base_client:get_with_retries:119 - Going too fast; sleeping 0.36s.
2023-06-19 08:15:39.914 | SUCCESS  | src.severa.base_client:get_with_retries:141 - HTTP/2 GET projects [0]: Response (100) in 0.64s.
2023-06-19 08:15:40.234 | DEBUG    | src.severa.base_client:get_with_retries:119 - Going too fast; sleeping 0.69s.
2023-06-19 08:15:40.945 | SUCCESS  | src.severa.base_client:get_with_retries:141 - HTTP/2 GET projects [0]: Response (100) in 0.31s.
2023-06-19 08:15:41.203 | DEBUG    | src.severa.base_client:get_with_retries:119 - Going too fast; sleeping 0.75s.
2023-06-19 08:15:41.975 | SUCCESS  | src.severa.base_client:get_with_retries:141 - HTTP/2 GET projects [0]: Response (100) in 0.25s.
2023-06-

In [92]:
x = pd.DataFrame(
    [
        {
            "name": p.name,
            "isclosed": p.isClosed,
            "internal": p.isInternal,
            "lastupdate": p.lastUpdatedDateTime,
        }
        for p in projs.values()
    ]
)

# 455
x[(x.lastupdate > arrow.utcnow().shift(months=-6).datetime)]
# 6 kk tuoreempia 138
x[~x.isclosed]
# avoimia 133
x[~x.internal]
# ei sisäisiä

x[(x.lastupdate > arrow.utcnow().shift(months=-4).datetime) & ~x.isclosed]
# 92

,name,isclosed,internal,lastupdate
103,Myllytie 4 tietomallikoordinointi,False,False,2023-03-03 16:25:36+02:00
156,"Leppäkorven koulu, tietomallikoordinointi",False,False,2023-06-15 11:34:31+03:00
182,"Rauman Karin Kampus, koordinointi",False,False,2023-03-10 13:24:49+02:00
183,"EKK - Espoon kulttuurikeskuksen teatteri, tiet...",False,False,2023-04-27 13:31:22+03:00
187,"Huippu, tietomallikoordinointi",False,False,2023-04-06 14:43:48+03:00
...,...,...,...,...
445,"Tapanilan pelastusasema, tietomallikoordinointi",False,False,2023-06-14 12:26:11+03:00
446,Heka Mestarintie 18 Haso Käskynhaltijantie 48,False,False,2023-06-08 16:04:30+03:00
447,Kymsote A-osa,False,False,2023-06-01 08:55:11+03:00
448,Tietomalliohjeet,False,False,2023-06-14 15:37:04+03:00


In [139]:
hours.loc[hours.date.isna() & (hours.id == "maximum"), "user"]

d = pd.DataFrame(
    {"date": pd.date_range("2023-05-01", "2023-05-30", freq="D", tz="utc")}
)
d["mask"] = d.date.map(Finland().is_working_day).map(int)
d

x = (
    hours.loc[hours.date.isna() & (hours.id == "maximum"), :]
    .drop("date", axis=1)
    .merge(d, how="cross")
)
x["value"] *= x["mask"]

In [157]:
x = src.severa.process.unravel_maximum(hours)
hours[hours.id == "maximum"]

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
5,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-20 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
6,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-21 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
7,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-22 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
8,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-23 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00


In [178]:
def unravel_maximum(data: pd.DataFrame) -> pd.DataFrame:
    min_date = data.loc[:, ["date", "start_date"]].min().min()
    max_date = data.loc[:, ["date", "end_date"]].max().max()
    print(min_date)
    print(max_date)
    dt_index = pd.date_range(min_date, max_date, freq="D", tz="utc")
    dates = pd.DataFrame({"date": dt_index})

    dates["workday_mask"] = dates.date.map(Finland().is_working_day).map(int)
    unraveled_subset = (
        data.loc[data.id == "maximum", :].drop("date", axis=1).merge(dates, how="cross")
    )
    unraveled_subset["value"] *= unraveled_subset["workday_mask"]

    # data.merge(unraveled_subset, on=["id", "user"])
    # data.loc[data.date.isna() & (data.id == "maximum"), :] = masking.drop(
    #    "workday_mask", axis=1
    # )

    return unraveled_subset


hours[hours.id == "maximum"]
hours[hours.id != "maximum"]

pd.concat(
    [unravel_maximum(hours).drop("workday_mask", axis=1), hours[hours.id != "maximum"]],
    ignore_index=True,
)

2020-03-15 00:00:00+00:00
2027-11-30 00:00:00+00:00


,user,value,id,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-16 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-17 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-18 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-19 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25915,0d100bc1-8376-7ac5-7bbb-055cdc20497d,73.5,saleswork,<NA>,f62568a5-36ce-5dc3-490d-a3a24031dba1,<NA>,<NA>,<NA>,2022-11-03 00:00:00+00:00,2025-12-01 00:00:00+00:00,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,2023-06-16 00:00:00+00:00,NaT
25916,2441bfbf-78f2-0173-b0a1-8634ce167a0d,22.0,saleswork,<NA>,e98dd28a-27be-a646-3141-bef922976214,<NA>,<NA>,<NA>,2023-04-25 00:00:00+00:00,2023-12-30 00:00:00+00:00,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-06-16 00:00:00+00:00,NaT
25917,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,12.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2023-08-01 00:00:00+00:00,2024-10-31 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT
25918,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,1.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2025-05-01 00:00:00+00:00,2025-06-30 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT


In [198]:
import random

dat = pd.DataFrame(
    {
        "id": list("AAABBB"),
        "value": [10, 8, 3, 7, 1, 1],
        "start_date": pd.Timestamp("2023-06-01"),
        "end_date": pd.Timestamp("2023-06-08"),
    }
)
dat.loc[1, "start_date"] = pd.Timestamp("2023-05-28")
dat.loc[2, "end_date"] = pd.Timestamp("2023-06-9")

dat["date"] = dat.apply(
    lambda x: pd.date_range(start=x["start_date"], end=x["end_date"]), axis=1
)
dat["_num_workdays"] = dat.apply(
    lambda x: Finland().get_working_days_delta(
        x["start_date"].date(), x["end_date"].date()
    ),
    axis=1,
)

dat = dat.explode("date").reset_index(drop=True)
dat["_is_workday"] = dat.date.map(Finland().is_working_day).map(int)
dat

hours[hours.date.isna()].sample(20)

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
516,2f365dcd-6ada-f6d8-8d73-4f393745e513,4.0,workhours,NaT,<NA>,efd040a7-b946-7f32-cdc7-5254d4707403,<NA>,False,e4a802ac-54f4-d3f2-6b0c-229f94118e7d,2023-06-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
545,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,25.0,workhours,NaT,<NA>,f1a3d420-e970-74be-7668-ab3104c00551,<NA>,False,1915e2b0-c434-0322-9175-f9454a283e70,2023-06-26 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
563,2441bfbf-78f2-0173-b0a1-8634ce167a0d,67.5,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
546,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,91.0,workhours,NaT,<NA>,2363278a-275f-ce2c-7a3b-5beea945c845,<NA>,False,67e159dc-1266-5c1c-51fa-228c95a1ea73,2022-10-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
538,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,8.0,workhours,NaT,<NA>,bf48e17f-7503-7ec5-87a8-25ad5cebd210,<NA>,False,079766cf-f3fd-80df-b9dd-e9bfc9a012e2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
555,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,27.0,workhours,NaT,<NA>,7dff98a8-da80-1006-4bb1-2bda3507c7d4,<NA>,False,fb38fbce-53da-62a6-57e2-5ff874ff5950,2023-05-12 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
530,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,10.0,workhours,NaT,<NA>,bc7710f2-37a0-d14b-fcd4-a9e4fe8a95dd,<NA>,False,39aad51a-e04d-7880-daa1-fef750b382d2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
490,eb8d73da-8c1c-7996-b085-332e777dff1a,2.0,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
514,2f365dcd-6ada-f6d8-8d73-4f393745e513,31.0,workhours,NaT,<NA>,c1e8f641-cadc-06ac-fa6a-334e8afaf814,<NA>,True,2b8f4b28-c55c-2326-66fd-6ad56748c059,2023-03-02 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
565,2441bfbf-78f2-0173-b0a1-8634ce167a0d,44.0,workhours,NaT,<NA>,dece01c7-5324-c4b4-2bab-497009174f92,<NA>,False,ff9a747b-a7de-1d49-4cca-adb8ffa19180,2023-04-07 00:00:00+00:00,2023-07-16 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00


In [301]:
y = src.severa.process.unravel(hours)

C:\Users\vireima\tie-dashboard\src\severa\process.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"_num_workdays"] = data.loc[:,:].apply(


In [302]:
y

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,xxx
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
1,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
2,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
3,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
4,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40840,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40841,2441bfbf-78f2-0173-b0a1-8634ce167a0d,0.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40842,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40843,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
